In [57]:
import pandas as pd
import numpy as np

In [58]:
import recsys.algorithm
recsys.algorithm.VERBOSE = True
from recsys.algorithm.factorize import SVD

In [87]:
df_book_ratings = pd.read_csv('BX-Book-Ratings.csv', sep=";", error_bad_lines=False)
df_books = pd.read_csv('BX-Books.csv', sep=";", error_bad_lines=False)
df_users = pd.read_csv('BX-Users.csv', sep=";", error_bad_lines=False)

Skipping line 6452: expected 8 fields, saw 9
Skipping line 43667: expected 8 fields, saw 10
Skipping line 51751: expected 8 fields, saw 9

Skipping line 92038: expected 8 fields, saw 9
Skipping line 104319: expected 8 fields, saw 9
Skipping line 121768: expected 8 fields, saw 9

Skipping line 144058: expected 8 fields, saw 9
Skipping line 150789: expected 8 fields, saw 9
Skipping line 157128: expected 8 fields, saw 9
Skipping line 180189: expected 8 fields, saw 9
Skipping line 185738: expected 8 fields, saw 9

Skipping line 209388: expected 8 fields, saw 9
Skipping line 220626: expected 8 fields, saw 9
Skipping line 227933: expected 8 fields, saw 11
Skipping line 228957: expected 8 fields, saw 10
Skipping line 245933: expected 8 fields, saw 9
Skipping line 251296: expected 8 fields, saw 9
Skipping line 259941: expected 8 fields, saw 9
Skipping line 261529: expected 8 fields, saw 9



In [89]:
def changeencode(data, cols):
    for col in data:
        data[col] = data[col].str.decode('iso-8859-1').str.encode('utf-8')
    return data 

In [60]:
# df_users = df_users.transpose()

In [61]:
df_book_ratings.head()
# df_book_ratings['ISBN'] = df_book_ratings['ISBN'].astype(str)
df_book_ratings.info()

# book ratings contain user id, book isbn, and rating.  No nullsb

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
User-ID        1149780 non-null int64
ISBN           1149780 non-null object
Book-Rating    1149780 non-null int64
dtypes: int64(2), object(1)
memory usage: 35.1+ MB


In [62]:
n = 250
top_n = df_book_ratings.ISBN.value_counts().index[:n]
df = df_book_ratings[df_book_ratings.ISBN.isin(top_n)]
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75509 entries, 8 to 1149772
Data columns (total 3 columns):
User-ID        75509 non-null int64
ISBN           75509 non-null object
Book-Rating    75509 non-null int64
dtypes: int64(2), object(1)
memory usage: 2.3+ MB


In [63]:
df_pivot = pd.pivot_table(df, values=["Book-Rating"],
        columns=["ISBN", "User-ID"],
        aggfunc=np.mean)
df_wide = df_pivot.unstack(-1)
df_wide.head()

User-ID                 9       14      16      26      44      51      \
            ISBN                                                         
Book-Rating 0060392452     NaN     NaN     NaN     NaN     NaN     NaN   
            0060502258     NaN     NaN     NaN     NaN     NaN     NaN   
            0060915544     NaN     NaN     NaN     NaN     NaN     NaN   
            0060928336     NaN     NaN     NaN     NaN     NaN     NaN   
            0060930535     NaN     NaN     NaN     NaN     NaN     NaN   

User-ID                 67      75      91      95       ...    278771  \
            ISBN                                         ...             
Book-Rating 0060392452     NaN     NaN     NaN     NaN   ...       NaN   
            0060502258     NaN     NaN     NaN     NaN   ...       NaN   
            0060915544     NaN     NaN     NaN     NaN   ...       NaN   
            0060928336     NaN     NaN     NaN     NaN   ...       NaN   
            0060930535     NaN     NaN     NaN     NaN   ...       NaN   

User-ID                 278773  278798  278800  278813  278828  278832  \
            ISBN                                                         
Book-Rating 0060392452     NaN     NaN     NaN     NaN     NaN     NaN   
            0060502258     NaN     NaN     NaN     NaN     NaN     NaN   
            0060915544     NaN     NaN     NaN     NaN     NaN     NaN   
            0060928336     NaN     NaN     NaN     NaN     NaN     NaN   
            0060930535     NaN     NaN     NaN     NaN     NaN     NaN   

User-ID                 278843  278844  278854  
            ISBN                                
Book-Rating 0060392452     NaN     NaN     NaN  
            0060502258     NaN     NaN     NaN  
            0060915544     NaN     NaN     NaN  
            0060928336     NaN     NaN     NaN  
            0060930535     NaN     NaN     NaN  

[5 rows x 23516 columns]

In [64]:
df_wide = df_wide.fillna(0)
ISBN = df_wide.index.levels[1]

In [65]:
from sklearn.metrics.pairwise import cosine_similarity


In [66]:
dists = cosine_similarity(df_wide)
dists

array([[ 1.        ,  0.01683897,  0.03071446, ...,  0.0166331 ,
         0.0296131 ,  0.0166903 ],
       [ 0.01683897,  1.        ,  0.        , ...,  0.03407822,
         0.02110809,  0.0177689 ],
       [ 0.03071446,  0.        ,  1.        , ...,  0.01634543,
         0.0593462 ,  0.        ],
       ..., 
       [ 0.0166331 ,  0.03407822,  0.01634543, ...,  1.        ,
         0.03397007,  0.01520413],
       [ 0.0296131 ,  0.02110809,  0.0593462 , ...,  0.03397007,
         1.        ,  0.02970534],
       [ 0.0166903 ,  0.0177689 ,  0.        , ...,  0.01520413,
         0.02970534,  1.        ]])

In [67]:
dists = pd.DataFrame(dists)
dists.columns = ISBN
dists.index = ISBN
dists.ix[0:10, 0:10]

ISBN,0060392452,0060502258,0060915544,0060928336,0060930535,0060934417,0060938455,0060959037,0060976845,0060977493
ISBN,,,,,,,,,,
0060392452,1.000000,0.016839,0.030714,0.017810,0.019459,0.015239,0.077219,0.008105,0.038340,0.028233
0060502258,0.016839,1.000000,0.000000,0.036336,0.017129,0.019925,0.010564,0.000000,0.048397,0.000000
0060915544,0.030714,0.000000,1.000000,0.039117,0.049460,0.053821,0.033138,0.076881,0.045099,0.046929
0060928336,0.017810,0.036336,0.039117,1.000000,0.065191,0.028421,0.016117,0.034256,0.131663,0.020998
0060930535,0.019459,0.017129,0.049460,0.065191,1.000000,0.020488,0.010031,0.026412,0.047523,0.046376
0060934417,0.015239,0.019925,0.053821,0.028421,0.020488,1.000000,0.012283,0.038548,0.031993,0.025512
0060938455,0.077219,0.010564,0.033138,0.016117,0.010031,0.012283,1.000000,0.011089,0.011204,0.017668
0060959037,0.008105,0.000000,0.076881,0.034256,0.026412,0.038548,0.011089,1.000000,0.022421,0.013870
0060976845,0.038340,0.048397,0.045099,0.131663,0.047523,0.031993,0.011204,0.022421,1.000000,0.011713


In [74]:
books_i_like = ['0060392452','0060930535']
dists[books_i_like].head()

ISBN,0060392452,0060930535
ISBN,,
0060392452,1.000000,0.019459
0060502258,0.016839,0.017129
0060915544,0.030714,0.049460
0060928336,0.017810,0.065191
0060930535,0.019459,1.000000


In [75]:
books_summed = dists[books_i_like].apply(lambda row: np.sum(row), axis=1)
books_summed

ISBN
0060392452    1.019459
0060502258    0.033968
0060915544    0.080175
0060928336    0.083001
0060930535    1.019459
0060934417    0.035727
0060938455    0.087249
0060959037    0.034517
0060976845    0.085863
0060977493    0.074609
0060987103    0.037464
0060987529    0.076779
0060987561    0.054545
0061009059    0.054759
006101351X    0.015143
0061097101    0.048159
0062502182    0.035490
0064407667    0.037046
0066214122    0.018551
0070212570    0.035116
0140067477    0.024414
014023313X    0.044599
0140244824    0.063035
014025448X    0.057399
014028009X    0.043141
0140293248    0.081700
014029628X    0.038120
0140298479    0.059010
0141000198    0.068632
0142000205    0.044788
                ...   
0684872153    0.024257
0684874350    0.048921
0743225406    0.026417
0743237188    0.075238
0743418174    0.038936
074343627X    0.013052
0767900383    0.067484
0767902521    0.097716
0767905180    0.029294
0786868716    0.046141
0786881852    0.026178
080410526X    0.027488
080410

In [80]:
def book_rec(ISBN_rec, k):
    book_rec_sum = dists[ISBN_rec].apply(lambda row: np.sum(row), axis=1)
    ranked_books_rec = book_rec_sum.order(ascending=False)
    ranked_books_rec_drop = ranked_books_rec.index[ranked_books_rec.index.isin(ISBN_rec)==False]
    print list(ranked_books_rec_drop[:k])

In [81]:
book_rec(['0060392452','0060930535'],10)

['0312195516', '0679781587', '0385484518', '0375727345', '0671038184', '0767902521', '0375706771', '0316601950', '0345361792', '0375703055']


In [82]:
import recsys.algorithm
recsys.algorithm.VERBOSE = True
from recsys.algorithm.factorize import SVD

In [83]:
svd = SVD()

In [91]:
changeencode(df_book_ratings,df_book_ratings.columns)

AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas

In [84]:
svd.load_data(df_book_ratings)

Loading          User-ID         ISBN  Book-Rating
0         276725   034545104X            0
1         276726   0155061224            5
2         276727   0446520802            0
3         276729   052165615X            3
4         276729   0521795028            6
5         276733   2080674722            0
6         276736   3257224281            8
7         276737   0600570967            6
8         276744   038550120X            7
9         276745    342310538           10
10        276746   0425115801            0
11        276746   0449006522            0
12        276746   0553561618            0
13        276746   055356451X            0
14        276746   0786013990            0
15        276746   0786014512            0
16        276747   0060517794            9
17        276747   0451192001            0
18        276747   0609801279            0
19        276747   0671537458            9
20        276747   0679776818            8
21        276747   0943066433            7
22 

TypeError: coercing to Unicode: need string or buffer, DataFrame found

In [85]:
k = 100
svd.compute(k=k, min_values=10, pre_normalize=None, mean_center=True, post_normalize=True)

ValueError: No data set. Matrix is empty!